In [1]:
import os
import json
#import yaml



def query_title(track_dict):
    
    title = track_dict['Title']
    artist = track_dict['Artist(s)']
    mix = track_dict['Mix']
    
    title_maker(title, artist, mix)
           
    return query

def title_maker(title, artist, mix):
    if mix.split(' ')[0] in title:
        query = "{} - {}".format(artist, title)    
    else:        
        query = "{} ({}) - {}".format(artist,title,mix)
    return query

def key_formatter(key):
    scale_type = key[-3:]
    root = key.split(scale_type)[0]

    if ' ' in root:
        root = root[:root.index(' ')]

    root = sharpen_flats(root)

    if root == 'B#':
        root = 'C'
    if root == 'E#':
        root = 'F' 

    key = '{} {}'.format(root, scale_type.lower())

    return key

def sharpen_flats(root):
    roots = ['C','D','E','F','G','A','B'] 
    if 'b' in root:
        natural_harmonic = root[:1]        
        idx = roots.index(natural_harmonic)
        sharpened_root = roots[idx-1]        
        root = "{}#".format(sharpened_root)      
    return root

## _queries to TechHouse_track_dicts

In [2]:
def queries_to_track_dicts(query_dicts):
    
    track_dicts = {}
    for track_dict in query_dicts.values(): 
        track_dicts[track_dict['Query']] = {'Duration': track_dict['Duration'],
                          'Key': track_dict['Key'],
                          'BPM': track_dict['BPM']}
    print(len(track_dicts))   
    return track_dicts

In [17]:
query_dict_name = 'traxsource_crawl_7500_10000_queries.json'
query_dict_path = '../data/metadata/traxsource/'+query_dict_name

with open(query_dict_path, 'r', encoding='utf-8') as infile:
    query_dicts = json.load(infile)
    
#print(query_dicts['2'])

track_dicts = queries_to_track_dicts(query_dicts)

2303


In [18]:
track_dicts_name = query_dict_name.split('_queries.json')[0] + '_track_dicts.json'
track_dicts_path = os.path.join('../data/metadata', track_dicts_name)

with open(track_dicts_path, 'w', encoding='utf-8') as outfile:
    json.dump(track_dicts, outfile,indent=4 )

In [ ]:
with open('../data/metadata/TechHouse_track_dicts_0.json', 'r', encoding='utf-8') as infile:
    track_dicts_0 = json.load(infile)
print(len(track_dicts_0))   

In [ ]:
track_dicts = {**track_dicts, **track_dicts_0}

print(len(track_dicts))

for track_dict in track_dicts.values():
    track_dict['Key'] = key_formatter(track_dict['Key'])

# check
for title in track_dicts.keys():
    
    track_dict = track_dicts[title]
    if track_dict['Key'].split(' ')[0] == 'B#' or track_dict['Key'].split(' ')[0] == 'E#':
        print('lol')

In [ ]:
with open('../data/metadata/TechHouse_track_dicts.json', 'w', encoding='utf-8') as outfile:
    json.dump(track_dicts, outfile,indent=4 )

In [ ]:
track_dicts = {**track_dicts, **track_dicts_0}

key_fixed_track_dicts = {}
for title in track_dicts.keys():
    
    track_dict = track_dicts[title]
    
    if track_dict['Key'].split(' ')[0] == 'B#' or track_dict['Key'].split(' ')[0] == 'E#':
        
        key_fixed_track_dicts[title] = {'Duration': track_dict['Duration'],
                                      'Key': key_formatter(track_dict['Key']),
                                      'BPM': track_dict['BPM']}
        
with open('../data/metadata/TechHouse_track_dicts_keyfixed.json', 'w', encoding='utf-8') as outfile:
    json.dump(key_fixed_track_dicts, outfile, indent=4 )

## Merger

In [ ]:
with open('../data/metadata/TechHouse_track_dicts.json', 'r', encoding='utf-8') as infile:
    TechHouse_track_dicts = json.load(infile)

In [ ]:
with open('../data/metadata/12_05_2021_TechHouse_Merged.json', 'r', encoding='utf-8') as infile:
    new_track_dicts = json.load(infile)

new_track_dict = {: track_dict for track_dict in new_track_dicts.values()}

In [ ]:
new_track_dict = {}

for track_dict in new_track_dicts.values():
    
    key = query_title(track_dict)
    
    new_track_dict[key] = {'Duration': track_dict['Duration'],
                          'Released': track_dict['Released'],
                           'Key': track_dict['Key'],
                           'BPM': track_dict['BPM']}

In [ ]:
new_track_dict

In [ ]:
merged_track_dicts = {**new_track_dict, **TechHouse_track_dicts}

In [ ]:
len(list(merged_track_dicts.keys()))

In [ ]:
with open('../data/metadata/TechHouse_track_dicts.json', 'r', encoding='utf-8') as infile:
    track_dicts = json.load(infile)

for track_dict in track_dicts.values():
    
    print(track_dict['Key'])

In [ ]:
with open('../data/metadata/TechHouse_track_dicts.json', 'w', encoding='utf-8') as outfile:
    json.dump(merged_track_dicts, outfile,indent=4 )

## YAML merge

In [ ]:
def artists_for_title(artists_list):

    artist_no = len(artists_list)
    if artist_no == 1:
        artists = artists_list[0]
    elif artist_no>1:
        artists = artists_list[0]
        for i in range(1,artist_no):
            artists += ", {}".format(artists_list[i])
    else:
        print('Not enough artist names!')
            
    return artists

def full_title_maker_YAML(title, artist, mix):
        
    if mix:
        return '{} - {} ({})'.format(artist, title, mix)
    else:
        return '{} - {}'.format(artist, title)

In [ ]:
with open('../data/metadata/traxsource/crawled_songs.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    yaml_list = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
track_dicts = {}
for track in yaml_list:
    
    try:
    
        title = list(track.keys())[0]
        x = track[title]

        artists = x['Artists']
        BPM = x['BPM']
        key = x['Key']
        label = x['Label']
        mix = x['Version']      

        artists = artists_for_title(artists)

        full_title = full_title_maker_YAML(title, artists, mix)

        track_dicts[full_title] = {'Key': key, 'BPM': BPM}
        
    except:
        
        print(track)

In [ ]:
len(track_dicts)

## Title List Maker

In [ ]:
#music_dir = os.path.join('E:','Music','Tech House')
music_dir = os.path.join('..','data','audio_clips')

In [ ]:
titles = []

for root,dirs,files in os.walk(music_dir):
    
    for file_name in files:
        
        if file_name.split('.')[-1] in '.mp3':
            
            track_title = file_name.split('.mp3')[0]
            
            titles.append(track_title)

In [ ]:
out_dir = os.path.join('..', 'data','bassline_extraction', 'ouz_tracks.txt')

with open(out_dir, 'w', encoding='utf-8') as outfile:
    outfile.write("\n".join(titles))